
## Constructing adminproject dashboard

In [3]:
#projadmin working template

#----------------------------------------------------------------------

# Import libraries
import re
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
token = open(".mapbox_token.txt").read()

#----------------------------------------------------------------------

#Import excel files to be used in app

adminproj = pd.read_excel('AdminMap_hydoprojects.xlsx')
spring_locations = pd.read_excel('spring_locations.xlsx')

#----------------------------------------------------------------------

# variables to be used in different places within app

heading={'color':'black', 'font-size':50,'text-Align':'center','font-weight': 'bold'}
subheading={'color':'black', 'font-weight': 'bold', 'text-Align':'left'}

options = [
    {'label': adminproj['Project Alias'][0], 'value':adminproj['Project ID'][0]},
    {'label': adminproj['Project Alias'][1], 'value':adminproj['Project ID'][1]},
    {'label': adminproj['Project Alias'][2], 'value':adminproj['Project ID'][2]},
    {'label': adminproj['Project Alias'][3], 'value':adminproj['Project ID'][3]},
    {'label': adminproj['Project Alias'][4], 'value':adminproj['Project ID'][4]},
    {'label': adminproj['Project Alias'][5], 'value':adminproj['Project ID'][5]},
    {'label': adminproj['Project Alias'][6], 'value':adminproj['Project ID'][6]}
]


#----------------------------------------------------------------------

# Initialize the app class

app = dash.Dash(__name__)
#https://stackoverflow.com/questions/50844844/python-dash-custom-css
#----------------------------------------------------------------------

# app layout

app.layout = html.Div([
    html.Div([
        html.H1(children=['Field Sites for Selected Project'], style={'textAlign':'center','color':heading['color']}),
        dcc.Graph(id='graph')],style={'width': '70%','padding-left':'2px','float':'right'}),
    html.Div([
    html.Label(children=['Select Project:'], style={'color':'black', 'font-weight': 'bold', 'text-Align':'left', 'padding-left':'6px','padding-top':'70px'}),
    dcc.Dropdown(id='project-picker', options = options)      
    ],  style={'width': '28%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 
               'border-color':'black','padding-top': '50px'}
    ),
    html.Div([
        html.Br(),
        html.Label(['Associated Publication or Report:'], style=subheading),
        html.Pre(id='publication', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'left',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
                ),
        html.Label(['Description of Project:'], style=subheading),
        html.Pre(id='description', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'left',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
                ),
        html.Label(['Associated Hydrology Objective'], style=subheading),
        html.Pre(id='objective', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'left',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
                ),
        html.Label(['Point of Contact'], style=subheading),
        html.Pre(id='poc', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'left',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
                ),
        html.Label(['Affiliated Organization'], style=subheading),
        html.Pre(id='affiliation', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'left',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
                )
    ], 
        style={'width': '28%', 'display': 'inline-block', 'vertical-align': 'right','border-bottom': 'solid 3px', 
               'border-color':'black','padding-left': '10px','padding-top': '6px'}
    )
])


#----------------------------------------------------------------------

# callback for Scattermapbox/graph

@app.callback(Output('graph',component_property='figure'),
              [Input('project-picker',component_property='value')
              ])

#filter spreadsheets selecting the springs in the project and create customdata variable to be used in hoverlabels
def springs_in_proj2(Project_ID):
    d = {adminproj['Project ID'][Project_ID-1] : adminproj['Sampling Locations'][Project_ID-1]}
    
    d1= re.split(', ', d[Project_ID])
   
    springs = spring_locations.set_index('Site Name')
    springs = springs.loc[d1]

    
    Site_Name = springs.index
    operator = springs['Operator']
    cont_data_collection = springs['Continuous Data Collection']
    cont_params = springs['Cont. Data Collection Params.']
    discr_params = springs['Discr. Data Collection Params.']
    
    customdata = pd.DataFrame(np.c_[Site_Name, operator, cont_data_collection, cont_params, discr_params])

    
#code for the actual scattermapbox plot
    
    traces = []
    for Project_ID in springs.index:
        
        
        
        traces.append(go.Scattermapbox(
            lon = springs['lon'], 
            lat = springs['lat'],
            marker = {'size': 8},  #name = str(springs['Project ID']),
            name = '',
            customdata = customdata,
            hovertemplate = 
            '<b>%{customdata[0]}</b><br> Operator: %{customdata[1]}<br> Active Continuous Data Collection: %{customdata[2]}<br> Cont. Collection: %{customdata[3]}<br> Disc. Collection: %{customdata[4]}<br>'
        ))
              
    return {
        'data':traces,
        'layout':go.Layout(
            showlegend = False,
            uirevision = 'foo',
            height=800,
            mapbox = {            
                'accesstoken': token,
                'style': "stamen-terrain", 
                'zoom': 8,
                'center': dict(lat=36,lon=-112)}
        )
    }


#----------------------------------------------------------------------

# callback for publication


@app.callback(
    Output('publication', 'children'),
    [Input('project-picker', 'value')])

def display_click_data(value):
    
    pub = adminproj['Associated Publication or Report'][value-1]
    return pub
    

#----------------------------------------------------------------------

# callback for project description


@app.callback(
    Output('description', 'children'),
    [Input('project-picker', 'value')])

def display_click_data(value):
    
    descr = adminproj['Description of Study'][value-1]
    return descr

#----------------------------------------------------------------------

# callback for hydrology objective


@app.callback(
    Output('objective', 'children'),
    [Input('project-picker', 'value')])

def display_click_data(value):
    
    objective = adminproj['Hydrology Objective'][value-1]
    return objective

#----------------------------------------------------------------------

# callback for point of contact


@app.callback(
    Output('poc', 'children'),
    [Input('project-picker', 'value')])

def display_click_data(value):
    
    poc = (adminproj['Point of Contact'][value-1])
    return poc

#----------------------------------------------------------------------

# callback for affiliation


@app.callback(
    Output('affiliation', 'children'),
    [Input('project-picker', 'value')])

def display_click_data(value):
    
    affiliation = (adminproj['Affiliation'][value-1])
    return affiliation



#----------------------------------------------------------------------------------------------------------
#Run the app

if __name__ == '__main__':
              app.run_server()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Sep/2020 11:16:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 11:16:29] "GET /prod HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 11:16:29] "GET /dev HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 11:16:29] "GET /assets/bootstrap.min.css?m=1599349867.28492 HTTP/1.1" 304 -
